Connect to Spotify API to generate token. Token is valid for 1 hour

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!curl -X POST "https://accounts.spotify.com/api/token" \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -d "grant_type=client_credentials&client_id=1471c52014e6448fb8b63f6d44621f84&client_secret=21d3008d56864a9fa88a9593644545e1"


{"access_token":"BQAMzNrq4AA1qpTxyUkG-Yfvx3l2iDYNIlBAtgTwwtrmzg7Z8DK5bIPYZBOftGZXziZKK49IlUJ_kUUwYKGh7v5SkJVNQG7vM8x_L4d8FmxEQLygDgM7","token_type":"Bearer","expires_in":3600}

In [ ]:
!pip install spotipy

import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 20.7 MB/s eta 0:00:00


In [ ]:
cid = '1471c52014e6448fb8b63f6d44621f84'
secret = '21d3008d56864a9fa88a9593644545e1'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
def call_playlist(creator, playlist_id):
    
    #step1

    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    #step2
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)

    #Step 3
        
    return playlist_df

In [ ]:
call_playlist("spotify","7vI0tN3yUn07dkK9T6p2pg")

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,ROSALÍA,RR,BESO,609E1JCInJncactoMmkDon,0.768,0.644,5,-6.671,0,0.1360,0.000837,0.1730,0.530,95.050,194543,4
1,KAROL G,MAÑANA SERÁ BONITO,TQG,0DWdj2oZMBFSzRsi2Cvfzf,0.720,0.630,4,-3.547,0,0.2770,0.000000,0.0936,0.607,179.974,199440,4
2,Yandel,Resistencia,Yandel 150,4FAKtPVycI4DxoOHC01YqD,0.783,0.729,6,-3.549,0,0.0691,0.000272,0.1000,0.580,167.968,216148,4
3,Marshmello,El Merengue,El Merengue,51FvjPEGKq2zByeeEQ43V9,0.775,0.677,8,-4.703,0,0.0442,0.005170,0.1120,0.698,124.011,189357,4
4,KAROL G,MAÑANA SERÁ BONITO,MIENTRAS ME CURO DEL CORA,6otePxalBK8AVa20xhZYVQ,0.523,0.477,0,-6.786,1,0.3920,0.000000,0.1460,0.570,79.921,164387,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Tokischa,Linda,Linda,1ahCrpeTt94LL7y1aXw0Y8,0.891,0.751,2,-3.150,1,0.1240,0.000598,0.1900,0.711,115.046,144019,4
96,J Balvin,JOSE,Una Nota,7iqOIHHEmu6Vhw25GgQ2eA,0.662,0.788,3,-4.550,0,0.2050,0.000000,0.0559,0.711,179.876,170760,4
97,Maluma,Sobrio,Sobrio,4HzqWEdAbzcVFDv0pvxV4w,0.762,0.770,5,-2.758,1,0.2230,0.000000,0.1500,0.631,177.998,201181,4
98,Mora,Volando (Remix),Volando - Remix,0G2zPzWqVjR68iNPmx2TBe,0.659,0.688,6,-4.649,1,0.0673,0.000000,0.1130,0.632,153.979,273474,4


In [ ]:
lst = call_playlist("spotify","7vI0tN3yUn07dkK9T6p2pg").track_name.values.tolist()

In [ ]:
len(lst)

100

Convert to spectrograms

SCRAP

In [ ]:
!pip install spotdl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import spotdl 
import shutil
import os

Mpath = '/content/Data'
os.mkdir(Mpath)

Compile the list

In [ ]:
!pip3 install git+https://github.com/mps-youtube/pafy.git -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/mps-youtube/pafy.git to /tmp/pip-req-build-2lm9lkbu
  Running command git clone --filter=blob:none --quiet https://github.com/mps-youtube/pafy.git /tmp/pip-req-build-2lm9lkbu
  Resolved https://github.com/mps-youtube/pafy.git to commit 0580c7247e74f4caa97154ad4fd6763e991198f8
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:


classes = [['jazz',
            'https://open.spotify.com/playlist/37i9dQZF1DWV7EzJMK2FUI?si=f4ee96d1194f4944', 
            'https://open.spotify.com/playlist/37i9dQZF1DXbITWG1ZJKYt?si=4b81e296affc4f2b',
            'https://open.spotify.com/playlist/37i9dQZF1DXdziGPHNE40t?si=69797eeaf17d44e3'], 
           ['classical', 
            'https://open.spotify.com/playlist/6wObnEPQ63a4kei1sEcMdH?si=f75cf2e39ccc4636',
            'https://open.spotify.com/playlist/37i9dQZF1DWWEJlAGA9gs0?si=2432fa81779f4386',
            'https://open.spotify.com/playlist/27Zm1P410dPfedsdoO9fqm?si=e0a178caf1d2467e',
            'https://open.spotify.com/playlist/37i9dQZF1DWUvHZA1zLcjW?si=f3c2dcb4da3d46f8',
            'https://open.spotify.com/playlist/37i9dQZF1DWUajed02NzWR?si=8548794b01184e6a',
            'https://open.spotify.com/playlist/37i9dQZF1DX0Aaer4Jzfgm?si=65dcc3a16c1a44af',
            'https://open.spotify.com/playlist/37i9dQZF1DX3lZNQOspLVY?si=882909648784416d'], 
           ['country',
            'https://open.spotify.com/playlist/2XlWXSTD1AYi4MIhhKnqCU?si=6e2050b99f574451',
            'https://open.spotify.com/playlist/4m2b9fA2s6h9zCdTIYj35d?si=c8ba3c7bc1ba4d4a',
            'https://open.spotify.com/playlist/37i9dQZF1DX13ZzXoot6Jc?si=e6f917216331442f',
            'https://open.spotify.com/playlist/27iJ7YElwDNiF2GDTPhrzf?si=0153fd206fb742bc',
            'https://open.spotify.com/playlist/17WrF3wsE02f0B711n9Dfh?si=c6c372cff1164844'], 
           ['disco', 
            'https://open.spotify.com/playlist/37i9dQZF1DX1MUPbVKMgJE?si=bf1f16472e4a4b73',
            'https://open.spotify.com/playlist/37i9dQZF1DX2GKumqRIZ7g?si=d0bb1180d9fb4432',
            'https://open.spotify.com/playlist/37i9dQZF1DWZXHVUlGM9Wp?si=bd046143963a44e2',
            'https://open.spotify.com/playlist/37i9dQZF1DX3jWba5xiDhV?si=02fbfcc79efd4721',
            'https://open.spotify.com/playlist/37i9dQZF1DXattPCMpISJh?si=25ce4db482cd4f39',
            'https://open.spotify.com/playlist/37i9dQZF1DWSvfPiFfb8Mi?si=9e4d810770d6436c',
            'https://open.spotify.com/playlist/37i9dQZF1DX9ZAByPehUEa?si=a1402e80a4024524',
            'https://open.spotify.com/playlist/37i9dQZF1DX0oFpWfPwcGv?si=fc40c05289794bfa',
            'https://open.spotify.com/playlist/37i9dQZF1DX0F8FtqyAjxC?si=438af571767f443e',
            'https://open.spotify.com/playlist/37i9dQZF1DX1tW4VlEfDSS?si=7d09b684829445c7',
            'https://open.spotify.com/playlist/37i9dQZF1DX1tW4VlEfDSS?si=9764ec6a4d70467d',
            'https://open.spotify.com/playlist/37i9dQZF1DXcAiMO5AYxE8?si=8dd0236dc1d54a82',
            'https://open.spotify.com/playlist/37i9dQZF1DX572PAi3rtlM?si=78a6cc2ecbf44dba',
            'https://open.spotify.com/playlist/37i9dQZF1DX11pUHJdXMnN?si=7245da955f8b427a',
            'https://open.spotify.com/playlist/37i9dQZF1DWVBWFXlHC6s7?si=c695b51722704eaa',
            'https://open.spotify.com/playlist/37i9dQZF1DX00RdhV73Dbe?si=40229f3888f94ebf',
            'https://open.spotify.com/playlist/37i9dQZF1DWTYPRTIhI2jZ?si=fd49c2897a474ce6'], 
           ['hip-hop',
            'https://open.spotify.com/playlist/3a9pVhH8G93BZ3OB9OTbhF?si=7d645f244d424aba'], 
           ['blues',
            'https://open.spotify.com/playlist/7KvP5E4IuBi6ctxgA9goAS?si=ac3117e60b5148ee',
            'https://open.spotify.com/playlist/37i9dQZF1DWWyCCtyRAvGr?si=be2341cf851e4884',
            'https://open.spotify.com/playlist/57Xv7xiPVlsYgUUqWFbVAR?si=bb1f52f3ec1142b0',
            'https://open.spotify.com/playlist/5TkTomPbQuSNDxdlWg2fCx?si=8cb21f13aab14857',
            'https://open.spotify.com/playlist/37i9dQZF1DXd9rSDyQguIk?si=f133ccb413614cb5',
            'https://open.spotify.com/playlist/3Z72yCEE7ejhoTr7C8liQy?si=df1d67df8e8140ea'], 
           ['metal',
            'https://open.spotify.com/playlist/6sKg0YErN93WCHpB3nIQYw?si=65f2d4b9d7314c5b'],
           ['pop',
            'https://open.spotify.com/playlist/6mtYuOxzl58vSGnEDtZ9uB?si=b91b1537d8eb438f',
            'https://open.spotify.com/playlist/2eVuLoCP74cegyax11zgEf?si=e6ee8527f66048ca',
            'https://open.spotify.com/playlist/2HL9uBibWwlRROmOvjK5tA?si=fa02a79ae4494d9a',
            'https://open.spotify.com/playlist/008G1BbvK1NQvbAV8MHvDz?si=8b67a54f81124ce0',
            'https://open.spotify.com/playlist/37i9dQZF1DXaPCIWxzZwR1?si=e5b39c4654ee4f1e',
            'https://open.spotify.com/playlist/37i9dQZF1DWSVpJBtEkFud?si=aeeb9bb696bf40f7',
            'https://open.spotify.com/playlist/37i9dQZF1DWXti3N4Wp5xy?si=0eb3135f18ea4656'],
           ['reggae',
            'https://open.spotify.com/playlist/3cxqjuczUihb20iEQmUfIC?si=55507d11a36c4959'],
           ['rock',
           'https://open.spotify.com/playlist/7KMtxb9kh3TtVvJTdLtJ2K?si=e96f73e6739c4953']]

download them

In [ ]:
for j in classes[0]:
  if j == classes[0][0]:
    dst = Mpath+'/'+j
    os.mkdir(dst)
  else:
    os.chdir(dst)
    url = j
    !spotdl $url

Streaming output truncated to the last 5000 lines.
Eas…                                                                            
High North Trio - For    Download Started   ━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--
Total                    421/551 complete   ━━━━━━━━━━━━━━━━╺━━━━━━  70% 0:25:02
Jonas Olofsson - It      Converting         ━━━━━━━━━━━━━━━━━━━━╺━━  88% 0:00:04
Never…                                                                          
Kendrettes - Cry Me a    Converting         ━━━━━━━━━━━━━━━━━╺━━━━━  76% 0:00:05
Riv…                                                                            
The Bob Davis Group -    Converting         ━━━━━━━━━━━━━━╸━━━━━━━━  65% 0:00:05
Eas…                                                                            
High North Trio - For    Download Started   ━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--
Total                    421/551 complete   ━━━━━━━━━━━━━━━━╺━━━━━━  70% 0:25:02
Jonas Olofsson - It      Converting         ━━━━━━━━━━━━━━

In [ ]:
shutil.move('content/Data/'+classes[0][0],'/content/drive/MyDrive/Project/NewData/')

In [ ]:
for j in classes[1]:
  if j == classes[1][0]:
    dst = Mpath+'/'+j
    os.mkdir(dst)
  else:
    os.chdir(dst)
    url = j
    !spotdl $url

In [ ]:
from google.colab import files
path = '/content/Data/classical'
for i in os.listdir(path):
  files.download('/content/Data/classical/'+i)

In [ ]:
shutil.move('/content/Data/classical','/content/drive/MyDrive/Project/NewData/')

In [ ]:
len(os.listdir('/content/Data/classical'))

In [ ]:
for j in classes[2]:
  if j == classes[2][0]:
    dst = Mpath+'/'+j
    os.mkdir(dst)
  else:
    os.chdir(dst)
    url = j
    !spotdl $url

In [ ]:
shutil.move('content/Data/'+classes[2][0],'/content/drive/MyDrive/Project/NewData/')

In [ ]:
for j in classes[3]:
  if j == classes[3][0]:
    dst = Mpath+'/'+j
    os.mkdir(dst)
  else:
    os.chdir(dst)
    url = j
    !spotdl $url

In [ ]:
shutil.move('content/Data/'+classes[3][0],'/content/drive/MyDrive/Project/NewData/')

In [ ]:
for j in classes[4]:
  if j == classes[4][0]:
    dst = Mpath+'/'+j
    os.mkdir(dst)
  else:
    os.chdir(dst)
    url = j
    !spotdl $url

In [ ]:
shutil.move('content/Data/'+classes[4][0],'/content/drive/MyDrive/Project/NewData/')

In [ ]:
for j in classes[5]:
  if j == classes[5][0]:
    dst = Mpath+'/'+j
    os.mkdir(dst)
  else:
    os.chdir(dst)
    url = j
    !spotdl $url

In [ ]:
shutil.move('content/Data/'+classes[5][0],'/content/drive/MyDrive/Project/NewData/')

In [ ]:
for j in classes[6]:
  if j == classes[6][0]:
    dst = Mpath+'/'+j
    os.mkdir(dst)
  else:
    os.chdir(dst)
    url = j
    !spotdl $url

In [ ]:
shutil.move('content/Data/'+classes[6][0],'/content/drive/MyDrive/Project/NewData/')

In [ ]:
for j in classes[7]:
  if j == classes[7][0]:
    dst = Mpath+'/'+j
    os.mkdir(dst)
  else:
    os.chdir(dst)
    url = j
    !spotdl $url

In [ ]:
shutil.move('content/Data/'+classes[7][0],'/content/drive/MyDrive/Project/NewData/')

In [ ]:
for j in classes[8]:
  if j == classes[8][0]:
    dst = Mpath+'/'+j
    os.mkdir(dst)
  else:
    os.chdir(dst)
    url = j
    !spotdl $url

In [ ]:
shutil.move('content/Data/'+classes[8][0],'/content/drive/MyDrive/Project/NewData/')

In [ ]:
for j in classes[9]:
  if j == classes[9][0]:
    dst = Mpath+'/'+j
    os.mkdir(dst)
  else:
    os.chdir(dst)
    url = j
    !spotdl $url

In [ ]:
shutil.move('content/Data/'+classes[9][0],'/content/drive/MyDrive/Project/NewData/')

split the audio to 30 secs each. Start from 45 to 1:15

In [ ]:
!pip isntall pydub
!apt-get ffmpeg